In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv', index_col='id')

In [3]:
train.shape

(57477, 8)

In [4]:
test.shape

(3, 3)

In [5]:
train.head(10)

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
id,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
292873,vicuna-13b,gpt-4-0314,"[""Construct a rap battle, in the style of Epic...","[""[Zeus]\nYo, it's the king of the gods on the...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...",0,1,0
313413,mixtral-8x7b-instruct-v0.1,vicuna-13b,"[""Why water is not used in bath tub?""]","[""Water is actually used in a bath tub. A bath...","[""Water is not used in a bath tub because it i...",1,0,0
370945,gemini-pro,claude-2.0,"[""\""Bacteria is life on Mars but a heartbeat i...","[""Dune""]","[""This quote seems to be referencing the debat...",0,1,0
441448,gpt-3.5-turbo-0613,vicuna-13b,"[""translate to russian the followig sentence ...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...",0,1,0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57477 entries, 30192 to 4294947231
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model_a         57477 non-null  object
 1   model_b         57477 non-null  object
 2   prompt          57477 non-null  object
 3   response_a      57477 non-null  object
 4   response_b      57477 non-null  object
 5   winner_model_a  57477 non-null  int64 
 6   winner_model_b  57477 non-null  int64 
 7   winner_tie      57477 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 3.9+ MB


In [7]:
train.columns

Index(['model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')

In [8]:
test.columns

Index(['prompt', 'response_a', 'response_b'], dtype='object')

In [9]:
features = ['prompt', 'response_a', 'response_b']

In [10]:
def target_row(row):
    if row['winner_model_a'] == 1:
        return 0

    elif row['winner_model_b'] == 1:
        return 1

    else:
        return 2
    

In [11]:
X = train[features]
y = train.apply(target_row, axis=1)

X_test = test.copy()

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, random_state=20, stratify=y)

X_train_text = X_train.astype(str).apply(lambda x: ' '.join(x), axis=1)
X_valid_text = X_valid.astype(str).apply(lambda x: ' '.join(x), axis=1)
X_test_text = test[features].astype(str).apply(lambda x: ' '.join(x), axis=1)

vect = TfidfVectorizer()
X_train = vect.fit_transform(X_train_text)
X_valid = vect.transform(X_valid_text)
X_test = vect.transform(X_test_text)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

lr = LogisticRegression(max_iter=500, solver='liblinear', random_state=42, multi_class='ovr').fit(X_train, y_train)


In [14]:
# Make predictions
y_pred_valid = lr.predict_proba(X_valid)
y_pred_test = lr.predict_proba(X_test)

# Calculate log loss (if needed)
valid_loss = log_loss(y_valid, y_pred_valid)
print(f"Validation Log Loss: {valid_loss}")

Validation Log Loss: 1.0992932222811667


In [15]:
sub = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')
sub['winner_model_a'] = y_pred_test[:, 0]
sub['winner_model_b'] = y_pred_test[:, 1]
sub['winner_tie'] = y_pred_test[:, 2]
sub.to_csv('submission.csv', index=False)
print(sub.head(10))

        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.197131        0.345800    0.457069
1   211333        0.469623        0.292566    0.237811
2  1233961        0.335789        0.466220    0.197991
